In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../../2024-03-11_myfitnesspal_587k_sample.csv")

In [4]:
df['Units'] = df['Units'].str.strip()


In [5]:
unit_counts = df['Units'].value_counts(normalize=True)
top_units = unit_counts.head(50)
top_units


Units
g              0.146185
cup            0.139082
oz             0.091821
tbsp           0.066672
slice          0.042862
egg            0.021248
slices         0.020599
ml             0.019821
serving s      0.018606
bar            0.017656
large          0.016330
container      0.015501
grams          0.014682
medium         0.014306
tsp            0.013260
packet         0.012100
fl             0.010740
sandwich       0.010494
serving        0.009257
scoop          0.008712
muffin         0.007574
bagel          0.006854
tablespoon     0.006412
pieces         0.006200
cups           0.006149
piece          0.005813
roll           0.004904
eggs           0.004389
package        0.004360
ounces         0.004180
small          0.003972
tbs            0.003900
banana         0.003766
biscuit        0.003487
tbsp s         0.003371
gr             0.003147
links          0.002966
tortilla       0.002927
donut          0.002848
tablespoons    0.002680
crackers       0.002659
cookie    

In [6]:
def replace_unit(u):
    return u if not u in list[1:] else list[0]

# lambda u: u if not u in list[1:] else list[0]

In [7]:
grams = ['g', 'grams', 'gr', 'gramm', 'grammes', 'gram', 'g s', 'gram s', 'gramme s']
slice = ['slice', 'slices', 'slice s']
piece = ['piece', 'pieces', 'piece s']
tablespoon = ['tablespoon', 'tablespoons', 'tbsps', 'tbsp', 'tbs', 'tbsp s', 'tbls', 'tbsl', 'tablespoon s']
teaspoon = ['teaspoon', 'teaspoons', 'tsp', 'tsp s', 'tsps', 'teaspoon s']
ounces = ['oz', 'ounces', 'oz s', 'ozs']
cups =['cup', 'cups', 'c', 'cup s']
eggs = ['egg', 'eggs', 'egg s']
scoops = ['scoop', 'scoops', 'scoop s']
serving = ['serving', 'servings', 'serving s']
cookies = ['cookie', 'cookies', 'biscuit', 'biscuits', 'biscuit s', 'cookie s']
dougnuts = ['doughnut', 'doughnuts', 'donut', 'donuts', 'donut s', 'doughnut s']
ml = ['ml', 'mls', 'ml s']

list_of_units = [grams, slice, piece, tablespoon, teaspoon, ounces, cups, eggs, scoops, serving, cookies, dougnuts, ml]


for list in list_of_units:
    df["Units"] = df["Units"].map(replace_unit)


In [8]:
df['Units'].nunique()


3594

In [34]:
from itertools import permutations

# make a table of scaling between 2 units: g & c
# mean all calories for grams, then mean all cups, then divide by each other
# the above must contain all permutations of the 2 units, so 2 in this case

avg_cals_per_gram = 20
avg_cals_per_cup = 200
avg_cals_per_oz = 40
avg_cals_per_serving = 800

list_of_permutations =[(avg_cals_per_gram, 'g'), (avg_cals_per_cup, 'cup'), (avg_cals_per_oz, 'oz'), (avg_cals_per_serving, 'serving')]

n = len(list_of_permutations)
k = 2

perm = permutations(range(1, n+1), k)
for p in perm:
    print(p)


(1, 2)
(1, 3)
(1, 4)
(2, 1)
(2, 3)
(2, 4)
(3, 1)
(3, 2)
(3, 4)
(4, 1)
(4, 2)
(4, 3)


In [25]:
avg_cals_per_gram

20

In [52]:
perm = permutations(range(1, n+1), k)

scaling_factor = []
for p in perm:
    # scale = p[0]/p[1]
    scale =round(list_of_permutations[p[0]-1][0]/list_of_permutations[p[1]-1][0],1)
    scaling_factor.append(scale)
print(scaling_factor)
# print(list_of_permutations[p[1]-1])

[0.1, 0.5, 0.0, 10.0, 5.0, 0.2, 2.0, 0.2, 0.1, 40.0, 4.0, 20.0]


In [53]:
list_of_permutations[p[0]-1][1]

'serving'

In [54]:
perm = permutations(range(1, n+1), k)
list_1 = []
list_2 = []
for p in perm:
    elem_1 = list_of_permutations[p[0]-1][1]
    elem_2 = list_of_permutations[p[1]-1][1]
    list_1.append(elem_1)
    list_2.append(elem_2)
df = pd.DataFrame({'column_A': list_1, 'column_B': list_2, 'scaling_factor': scaling_factor})
print(df)

   column_A column_B  scaling_factor
0         g      cup             0.1
1         g       oz             0.5
2         g  serving             0.0
3       cup        g            10.0
4       cup       oz             5.0
5       cup  serving             0.2
6        oz        g             2.0
7        oz      cup             0.2
8        oz  serving             0.1
9   serving        g            40.0
10  serving      cup             4.0
11  serving       oz            20.0


3617